In [ ]:
import gradio as gr

import torch
from PIL import Image, ImageFilter



from diffusers import AutoPipelineForInpainting, ControlNetModel
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler, UniPCMultistepScheduler
from diffusers import StableDiffusionPipeline as SDPipeline

In [ ]:
pipe = AutoPipelineForInpainting.from_pretrained(
  # "runwayml/stable-diffusion-inpainting",
  # "stable-diffusion-v1-5/stable-diffusion-inpainting",
  "stabilityai/stable-diffusion-2-inpainting",
  safety_checker=None,
  torch_dtype=torch.float16,
  variant="fp16",
).to("cuda")

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
import numpy as np

In [ ]:
im = Image.open("./base.jpg")
mask = Image.open("./mask.jpg")#.filter(ImageFilter.GaussianBlur((2,2)))
iw,ih = im.size

mask = mask.resize((mask.width, ih))

display(
  Image.fromarray(
    np.concatenate(
      [np.array(im), np.array(mask)],
      axis=1
    )
  ).resize((iw, ih//2))
)

In [ ]:
prompt = "person facing away from the camera, we don't see their faces, we only see the back of their heads and hair"
negative_prompt = "eyes, mask, mouth, nose, forehead"

output = pipe(prompt=prompt, negative_prompt=negative_prompt, image=im, mask_image=mask,
              num_inference_steps=80, guidance_scale=32.0, strength=0.9,
              width=iw, height=ih)
display(output.images[0])